In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bidi-reordered-dataset/bidi_reordered_attacked.csv


In [2]:
!sudo apt-get update -qq
!sudo apt-get install -qq unifont tesseract-ocr libtesseract-dev libleptonica-dev pkg-config
!pip install pillow tesserocr -q
# Arial Unicode font works best for OCR + Unicode support
!mkdir -p /usr/share/fonts/truetype/arialuni/
!curl -s https://raw.githubusercontent.com/kaienfr/Font/master/font/ARIALUNI.TTF --output /usr/share/fonts/truetype/arialuni/arialuni.ttf
!chmod 644 /usr/share/fonts/truetype/arialuni/arialuni.ttf
!fc-cache -f
!cp unifont.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 110195 files and directories currently installed.)
Preparing to unpack .../0-libarchive-dev_3.4.0-2ubuntu1.2_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.4.0-2ubuntu1.2) ...
Selecting previously unselected package libleptonica-dev:amd64.
Preparing to unpack .../1-libleptonica-dev_1.79.0-1_amd64.deb ...
Unpacking libleptonica-dev:amd64 (1.79.0-1) ...
Selecting previously unselected package libtesseract-dev:amd64.
Preparing to unpack .../2-libtesseract-dev_4.1.1-2build2_amd64.deb ...
Unpacking libtesseract-dev:amd64 (4.1.1-2build2) ...
Selecting previously unselected package psf-unifont.
Preparing to unpack .../3-psf-unifont_1%3a12.0.01-2_all.deb ...
Unpacking psf-unifont (1:12.0.01-2) ...
Selecting previously unselected package ttf-unifont.
Preparing to unpack .../4-ttf-unifont_1%3a12.0.01-2_all.deb ...
Unpacking ttf-unifont (1:12.0.01-2) ...
Selecting previously unselected package xfonts-unifont.
Pre

In [19]:
# !export TESSDATA_PREFIX='/usr/local/share/'

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [20]:
!sudo apt-get install tesseract-ocr-eng  #for english
!sudo apt-get install tesseract-ocr-tam  #for tamil
!sudo apt-get install tesseract-ocr-deu  #for deutsch (German)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1).
0 upgraded, 0 newly installed, 0 to remove and 60 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-tam is already the newest version (1:4.00~git30-7274cfa-1).
0 upgraded, 0 newly installed, 0 to remove and 60 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-deu is already the newest version (1:4.00~git30-7274cfa-1).
0 upgraded, 0 newly installed, 0 to remove and 60 not upgraded.


In [24]:
!git clone --recurse-submodules https://github.com/tesseract-ocr/tessdata_fast.git 2> /dev/null || (cd tessdata_fast; git pull)
!cp tessdata_fast/*.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

Submodule path 'tessconfigs': checked out '3decf1c8252ba6dbeef0bf908f4b0aab7f18d113'


In [12]:
import tesserocr
from tesserocr import PyTessBaseAPI
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import torchvision.transforms as transforms
import torch
import numpy as np
from PIL import Image
from math import ceil
from typing import List
from tqdm import tqdm


In [16]:
# should be a multiple of 2
font_size = 16
# arialuni.ttf best for tesserocr, unifont.ttf best for unicode coverage
font_file = "unifont.ttf"

font = ImageFont.truetype(font_file, font_size)     
_d = ImageDraw.Draw(Image.new("1", (0, 0), 1))

def draw(text: str) -> Image:
    """
    This function creates a binary image with text drawn on it.

    Args:
      text: The text string to draw on the image.

    Returns:
      A PIL Image object containing the drawn text.
    """
    # Get the width of the text using textlength
    width = int(ceil(_d.textlength(text, font=font)))  # Round up width to ensure integer
    # Set a reasonable height based on font size or desired aspect ratio
    height = font_size + 4  # Adjust height as needed

    # Create the image with the calculated size
    img = Image.new("1", (width, height), 1)
    d = ImageDraw.Draw(img)
    d.text((0, 0), text, font=font, fill=0)
    return img


def ocr(img: Image) -> str:
    # Tesseract works better if you fine-tune it on your font.
    # If you do this, specify the new model with `lang=`
    return tesserocr.image_to_text(img).rstrip('\n')

def draw_ocr(text: str) -> str:
    return ocr(draw(text))

def multiline_draw_ocr(text: str) -> str:
    """
    This function splits text by newline characters, performs OCR on each line
    using draw_ocr, and concatenates the results.

    Args:
      text: The text string potentially containing multiple lines.

    Returns:
      A string containing the OCR results for each line, concatenated together.
    """
    # Split the text into lines
    lines = text.splitlines()

    # Perform OCR on each line and store results
    ocr_results = []
    with PyTessBaseAPI() as api:
        for line in lines:
            try:
                ocr_result = draw_ocr(line)
                ocr_results.append(ocr_result.strip())  # Remove potential leading/trailing whitespace
            except Exception as e:
                print(f"Error occurred for line: {line}. Error: {e}")
                ocr_results.append("")  # Add an empty string for lines with errors

    # Concatenate the OCR results
    return "\n".join(ocr_results)


In [21]:
import pandas as pd

# Function to perform OCR on text using draw_ocr
def perform_ocr(text):
    try:
        # Attempt OCR using draw_ocr function
        ocr_result = multiline_draw_ocr(text)
        return ocr_result
    except Exception as e:
        # Handle any potential errors during OCR
        print(f"Error occurred for text: {text}. Error: {e}")
        return np.nan  # Or any other appropriate value for missing data
def process_dataset(filepath):
    # Load the dataset using pandas
    data = pd.read_csv(filepath)
    # Sample 5 random rows (indices) without replacement
    sample_indices = random.sample(range(len(data)), 5)

    # Get the sampled data
    sample_data = data.iloc[sample_indices]
    # Get the column containing the text for OCR
    text_column = "Reordered Text"  # Assuming the label is "Reordered Text"
    
    # Create empty lists to store data
    ocr_results = []
    original_texts = []

    # Iterate through each line (text) and perform OCR
    for text in tqdm(sample_data[text_column]):
        ocr_result = perform_ocr(text)
        ocr_results.append(ocr_result)
        original_texts.append(text)

    # Create a DataFrame with desired columns
    df = pd.DataFrame({"Reordered Text": original_texts, "OCR Result": ocr_results})

    # Save the DataFrame to a CSV file
    output_file = "processed_dataset.csv"  # Modify the output file name as needed
    df.to_csv(output_file, index=False)
    print(f"Processed data saved to: {output_file}")


In [29]:
# Replace "/kaggle/input/bidi-reordered-dataset/bidi_reordered_attacked.csv" with your actual file path
dataset_path = "/kaggle/input/bidi-reordered-dataset/bidi_reordered_attacked.csv"
process_dataset(dataset_path)

  0%|          | 0/5 [00:00<?, ?it/s]Error opening data file ./eng.traineddata
Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory.
Failed loading language 'eng'
Tesseract couldn't load any languages!
Error opening data file ./eng.traineddata
Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory.
Failed loading language 'eng'
Tesseract couldn't load any languages!
Error opening data file ./eng.traineddata
Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory.
Failed loading language 'eng'
Tesseract couldn't load any languages!
Error opening data file ./eng.traineddata
Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory.
Failed loading language 'eng'
Tesseract couldn't load any languages!
Error opening data file ./eng.traineddata
Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory.
10

Error occurred for text: ‭⁦‮⁦‭⁦‮⁦p⁩⁦o⁩‬⁩‬⁩⁦Pe⁩‬⁩‬le ‭⁦‮⁦n⁩⁦i⁩‬⁩‬ Ad‭⁦‮⁦u⁩⁦j⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦h⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦i⁩⁦n⁩‬⁩‬⁩⁦ma⁩‬⁩‬⁩‬⁩‬av‭⁦‮⁦‭⁦‮⁦u⁩⁦q⁩‬⁩‬⁩⁦e‭⁦‮⁦a⁩⁦ ⁩‬⁩‬c⁩‬⁩‬ir‭⁦‮⁦‭⁦‮⁦i⁩⁦k⁩‬⁩‬⁩⁦e‭⁦‮⁦ ⁩⁦d⁩‬⁩‬s⁩‬⁩‬lls‭⁦‮⁦t⁩⁦ ⁩‬⁩‬‭⁦‮⁦ ⁩⁦o⁩‬⁩‬e‭⁦‮⁦‭⁦‮⁦ ⁩⁦e⁩‬⁩‬⁩⁦ng‭⁦‮⁦g⁩⁦a⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦u⁩⁦b⁩‬⁩‬⁩⁦‭⁦‮⁦n⁩⁦i⁩‬⁩‬ ⁩‬⁩‬sin‭⁦‮⁦‭⁦‮⁦ ⁩⁦.⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦s⁩⁦s⁩‬⁩‬⁩⁦e⁩‬⁩‬⁩‬⁩‬I‭⁦‮⁦ ⁩⁦n⁩‬⁩‬fact,‭⁦‮⁦s⁩⁦ ⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦e⁩⁦l⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦o⁩⁦e⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦e⁩⁦m⁩‬⁩‬⁩⁦o⁩‬⁩‬ p⁩‬⁩‬p⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦t⁩⁦ ⁩‬⁩‬⁩⁦ in⁩‬⁩‬h‭⁦‮⁦‭⁦‮⁦i⁩⁦s⁩‬⁩‬⁩⁦e ⁩‬⁩‬t‭⁦‮⁦a⁩⁦u⁩‬⁩‬tion‭⁦‮⁦a⁩⁦ ⁩‬⁩‬r‭⁦‮⁦‭⁦‮⁦s⁩⁦a⁩‬⁩‬⁩⁦e‭⁦‮⁦s⁩⁦ ⁩‬⁩‬urp⁩‬⁩‬‭⁦‮⁦i⁩⁦s⁩‬⁩‬ng ‭⁦‮⁦h⁩⁦t⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦a⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦i⁩⁦e⁩‬⁩‬r⁩‬⁩‬c‭⁦‮⁦u⁩⁦t⁩‬⁩‬‭⁦‮⁦l⁩⁦a⁩‬⁩‬ ‭⁦‮⁦‭⁦‮⁦a⁩⁦p⁩‬⁩‬⁩⁦‭⁦‮⁦a⁩⁦c⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦i⁩⁦b⁩‬⁩‬‭⁦‮⁦i⁩⁦l⁩‬⁩‬‭⁦‮⁦i⁩⁦t⁩‬⁩‬e‭⁦‮⁦ ⁩⁦s⁩‬⁩‬by ena‭⁦‮⁦l⁩⁦b⁩‬⁩‬i‭⁦‮⁦‭⁦‮⁦r⁩⁦i⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦h⁩⁦t⁩‬⁩‬⁩⁦ng ⁩‬⁩‬e⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦a⁩⁦b⁩‬⁩‬⁩⁦ h‭⁦‮⁦s⁩⁦u⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦ ⁩⁦s⁩‬⁩‬⁩⁦nd⁩‬⁩‬t‭⁦‮⁦ ⁩⁦o⁩‬⁩‬di‭⁦‮⁦ ⁩⁦g⁩‬⁩‬de‭⁦‮⁦p⁩⁦e⁩‬⁩‬e‭⁦‮⁦‭⁦‮⁦o⁩⁦ ⁩‬⁩‬⁩⁦r‭⁦‮⁦i⁩⁦ ⁩‬⁩‬n‭⁦‮⁦o⁩⁦t⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦i⁩⁦l⁩‬⁩‬⁩⁦n⁩‬⁩‬n‭⁦‮⁦ ⁩⁦e⁩‬⁩‬‭⁦‮⁦p⁩⁦s⁩‬

In [20]:
from PIL import Image
import pytesseract
import numpy as np
from typing import List
from tqdm import tqdm
import cv2


# Assuming Tesseract is installed and configured
# pytesseract.pytesseract.tesseract_cmd = r'/path/to/tesseract.exe'  # Replace with your Tesseract path

# should be a multiple of 2
font_size = 16
# arialuni.ttf best for tesserocr, unifont.ttf best for unicode coverage
font_file = "unifont.ttf"

font = ImageFont.truetype(font_file, font_size)  

def multiline_draw(text: str) -> str:
    """
    This function splits text by newline characters, performs OCR on each line
    using pytesseract, and concatenates the results.

    Args:
      text: The text string potentially containing multiple lines.

    Returns:
      A string containing the OCR results for each line, concatenated together.
    """
    # Split the text into lines
    lines = text.splitlines()

    # Perform OCR on each line and store results
    ocr_results = []
    for line in lines:
        try:
            # Convert PIL image to OpenCV format (BGR) for pytesseract
            img = np.array(Image.new("RGB", (len(line) * font_size, font_size), color=(255, 255, 255)))
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Import cv2 if not already imported

            # Draw text on the image (optional)
            cv2.putText(img, line, (0, font_size), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
#             cv2.imshow('image',img)
#             cv2.waitKey(0)
            # Perform OCR using pytesseract
            ocr_result = pytesseract.image_to_string(img)  # psm 6 for single line mode
            ocr_results.append(ocr_result.strip())  # Remove potential leading/trailing whitespace
        except Exception as e:
            print(f"Error occurred for line: {line}. Error: {e}")
            ocr_results.append("")  # Add an empty string for lines with errors

    # Concatenate the OCR results
    return "\n".join(ocr_results)

def process_dataset(filepath):
    # Load the dataset using pandas (assuming a CSV file)
    data = pd.read_csv(filepath)
    # Sample 5 random rows (indices) without replacement
    sample_indices = random.sample(range(len(data)), 5)

    # Get the sampled data
    sample_data = data.iloc[sample_indices]
    # Get the column containing the text for OCR
    text_column = "Reordered Text"  # Assuming the label is "Reordered Text"

    # Create empty lists to store data
    ocr_results = []
    original_texts = []

    # Use tqdm for progress bar
    for text in tqdm(sample_data[text_column]):
        ocr_result = multiline_draw(text)
        ocr_results.append(ocr_result)
        original_texts.append(text)

    # Create a DataFrame with desired columns
    df = pd.DataFrame({"Reordered Text": original_texts, "OCR Result": ocr_results})

    # Save the DataFrame to a CSV file
    output_file = "processed_dataset.csv"  # Modify the output file name as needed
    df.to_csv(output_file, index=False)
    print(f"Processed data saved to: {output_file}")

# Replace "/kaggle/input/bidi-reordered-dataset/bidi_reordered_attacked.csv" with your actual file path
dataset_path = "/kaggle/input/bidi-reordered-dataset/bidi_reordered_attacked.csv"
process_dataset(dataset_path)


 40%|████      | 2/5 [00:07<00:08,  2.99s/it]

Error occurred for line: ‭⁦‮⁦‭⁦‮⁦I⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦t⁩⁦n⁩‬⁩‬⁩⁦E⁩‬⁩‬eb‭⁦‮⁦e⁩⁦b⁩‬⁩‬⁩‬⁩‬ntern‭⁦‮⁦‭⁦‮⁦o⁩⁦i⁩‬⁩‬⁩⁦‭⁦‮⁦t⁩⁦a⁩‬⁩‬⁩‬⁩‬n‭⁦‮⁦l⁩⁦a⁩‬⁩‬ Air‭⁦‮⁦o⁩⁦p⁩‬⁩‬‭⁦‮⁦t⁩⁦r⁩‬⁩‬ ‭⁦‮⁦a⁩⁦w⁩‬⁩‬s‭⁦‮⁦c⁩⁦ ⁩‬⁩‬lose‭⁦‮⁦‭⁦‮⁦ ⁩⁦s⁩‬⁩‬⁩⁦d‭⁦‮⁦a⁩⁦ ⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦m⁩⁦ ⁩‬⁩‬⁩⁦a⁩‬⁩‬‭⁦‮⁦a⁩⁦e⁩‬⁩‬s‭⁦‮⁦‭⁦‮⁦ ⁩⁦o⁩‬⁩‬⁩⁦ur‭⁦‮⁦ ⁩⁦e⁩‬⁩‬t⁩‬⁩‬c‭⁦‮⁦‭⁦‮⁦ ⁩⁦e⁩‬⁩‬⁩⁦ur‭⁦‮⁦ ⁩⁦b⁩‬⁩‬th⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦ ⁩⁦d⁩‬⁩‬⁩⁦sp‭⁦‮⁦e⁩⁦r⁩‬⁩‬a⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦o⁩⁦r⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦o⁩⁦c⁩‬⁩‬⁩⁦o‭⁦‮⁦ ⁩⁦f⁩‬⁩‬⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦a⁩⁦n⁩‬⁩‬vi‭⁦‮⁦u⁩⁦r⁩‬⁩‬‭⁦‮⁦.⁩⁦s⁩‬⁩‬Ma‭⁦‮⁦e⁩⁦k⁩‬⁩‬‭⁦‮⁦s⁩⁦ ⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦o⁩⁦y⁩‬⁩‬⁩⁦ure ⁩‬⁩‬u‭⁦‮⁦d⁩⁦ ⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦a⁩⁦o⁩‬⁩‬⁩⁦‭⁦‮⁦w⁩⁦o⁩‬⁩‬nl⁩‬⁩‬‭⁦‮⁦ ⁩⁦d⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦p⁩⁦a⁩‬⁩‬⁩⁦th‭⁦‮⁦ ⁩⁦e⁩‬⁩‬⁩‬⁩‬p‭⁦‮⁦‭⁦‮⁦o⁩⁦r⁩‬⁩‬⁩⁦ f⁩‬⁩‬m‭⁦‮⁦‭⁦‮⁦ ⁩⁦e⁩‬⁩‬⁩⁦ ‭⁦‮⁦h⁩⁦t⁩‬⁩‬⁩‬⁩‬a‭⁦‮⁦‭⁦‮⁦.⁩⁦e⁩‬⁩‬⁩⁦‭⁦‮⁦p⁩⁦p⁩‬⁩‬ ‭⁦‮⁦t⁩⁦s⁩‬⁩‬or⁩‬⁩‬ ‭⁦‮⁦‭⁦‮⁦y⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦o⁩⁦s⁩‬⁩‬⁩⁦‭⁦‮⁦l⁩⁦A⁩‬⁩‬⁩‬⁩‬⁩‬⁩‬o‭⁦‮⁦ ⁩⁦u⁩‬⁩‬c‭⁦‮⁦n⁩⁦a⁩‬⁩‬ ch‭⁦‮⁦c⁩⁦e⁩‬⁩‬‭⁦‮⁦ ⁩⁦k⁩‬⁩‬t‭⁦‮⁦‭⁦‮⁦a⁩⁦h⁩‬⁩‬⁩⁦‭⁦‮⁦e⁩⁦h⁩‬⁩‬‭⁦‮⁦c⁩⁦ ⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦i⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦e⁩⁦t⁩‬⁩‬⁩⁦rac⁩‬⁩‬rs⁩‬⁩‬n ga‭⁦‮⁦e⁩⁦m⁩‬⁩‬ ‭⁦‮⁦‭⁦‮⁦c⁩⁦i⁩‬⁩‬⁩⁦wh⁩‬⁩‬h ar‭⁦‮⁦ ⁩⁦e⁩‬⁩‬‭⁦‮⁦

 60%|██████    | 3/5 [00:07<00:03,  1.72s/it]

Error occurred for line: ‭⁦‮⁦‭⁦‮⁦e⁩⁦p⁩‬⁩‬⁩⁦‭⁦‮⁦i⁩⁦F⁩‬⁩‬‭⁦‮⁦t⁩⁦f⁩‬⁩‬y ⁩‬⁩‬r‭⁦‮⁦‭⁦‮⁦t⁩⁦n⁩‬⁩‬⁩⁦ ‭⁦‮⁦e⁩⁦c⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦s⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦o⁩⁦ ⁩‬⁩‬f⁩‬⁩‬e‭⁦‮⁦d⁩⁦e⁩‬⁩‬s ‭⁦‮⁦o⁩⁦s⁩‬⁩‬‭⁦‮⁦d⁩⁦l⁩‬⁩‬ o‭⁦‮⁦‭⁦‮⁦a⁩⁦g⁩‬⁩‬⁩⁦n t‭⁦‮⁦e⁩⁦h⁩‬⁩‬ U⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦n⁩⁦a⁩‬⁩‬⁩⁦‭⁦‮⁦d⁩⁦n⁩‬⁩‬⁩‬⁩‬ ma‭⁦‮⁦k⁩⁦r⁩‬⁩‬et‭⁦‮⁦‭⁦‮⁦u⁩⁦o⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦c⁩⁦ ⁩‬⁩‬⁩⁦ are⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦f⁩⁦r⁩‬⁩‬⁩⁦‭⁦‮⁦t⁩⁦n⁩‬⁩‬e⁩‬⁩‬eit‭⁦‮⁦ ⁩⁦.⁩‬⁩‬‭⁦‮⁦a⁩⁦T⁩‬⁩‬‭⁦‮⁦g⁩⁦r⁩‬⁩‬e‭⁦‮⁦‭⁦‮⁦y⁩⁦b⁩‬⁩‬⁩⁦t‭⁦‮⁦d⁩⁦e⁩‬⁩‬ ⁩‬⁩‬ the same Nig‭⁦‮⁦r⁩⁦e⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦a⁩⁦c⁩‬⁩‬⁩⁦i‭⁦‮⁦n⁩⁦a⁩‬⁩‬ ⁩‬⁩‬rte‭⁦‮⁦‭⁦‮⁦f⁩⁦ ⁩‬⁩‬⁩⁦ls‭⁦‮⁦a⁩⁦ ⁩‬⁩‬‭⁦‮⁦d⁩⁦n⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦r⁩⁦a⁩‬⁩‬⁩⁦ake m⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦r⁩⁦e⁩‬⁩‬⁩⁦ket⁩‬⁩‬s‭⁦‮⁦ ⁩⁦,⁩‬⁩‬‭⁦‮⁦o⁩⁦m⁩‬⁩‬s‭⁦‮⁦‭⁦‮⁦k⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦w⁩⁦ ⁩‬⁩‬⁩⁦t‭⁦‮⁦y⁩⁦l⁩‬⁩‬⁩‬⁩‬ith⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦d⁩⁦e⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦l⁩⁦w⁩‬⁩‬⁩⁦‭⁦‮⁦o⁩⁦n⁩‬⁩‬⁩‬⁩‬⁩‬⁩‬ge‭⁦‮⁦o⁩⁦ ⁩‬⁩‬‭⁦‮⁦ ⁩⁦f⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦ ⁩⁦'⁩‬⁩‬⁩⁦‭⁦‮⁦e⁩⁦s⁩‬⁩‬‭⁦‮⁦d⁩⁦e⁩‬⁩‬s⁩‬⁩‬i‭⁦‮⁦e⁩⁦d⁩‬⁩‬n‭⁦‮⁦i⁩⁦t⁩‬⁩‬t‭⁦‮⁦‭⁦‮⁦r⁩⁦o⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦,⁩‬⁩‬⁩⁦y⁩‬⁩‬ign⁩‬⁩‬an‭⁦‮⁦e⁩⁦c⁩‬⁩‬ and‭⁦‮⁦‭⁦‮⁦c⁩⁦a⁩‬⁩‬⁩⁦ l⁩‬⁩‬k ‭⁦‮⁦f⁩⁦o⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦t⁩⁦i⁩‬⁩‬⁩⁦ ‭⁦‮⁦b⁩⁦a⁩‬⁩‬‭⁦‮⁦l⁩⁦i⁩‬⁩‬⁩‬⁩‬y‭⁦‮⁦‭⁦‮⁦

100%|██████████| 5/5 [00:18<00:00,  3.64s/it]

Processed data saved to: processed_dataset.csv


In [9]:
import random
def process_dataset(filepath):
    # Load the dataset using pandas (assuming a CSV file)
    data = pd.read_csv(filepath)
    # Sample 5 random rows (indices) without replacement
    sample_indices = random.sample(range(len(data)), 5)

    # Get the sampled data
    sample_data = data.iloc[sample_indices]
    # Get the column containing the text for OCR
    text_column = "Reordered Text"  # Assuming the label is "Reordered Text"

    # Create empty lists to store data
    ocr_results = []
    original_texts = []

    # Use tqdm for progress bar
    for text in tqdm(sample_data[text_column]):
        ocr_result = multiline_draw(text)
        ocr_results.append(ocr_result)
        original_texts.append(text)

    # Create a DataFrame with desired columns
    df = pd.DataFrame({"Reordered Text": original_texts, "OCR Result": ocr_results})

    # Save the DataFrame to a CSV file
    output_file = "processed_dataset.csv"  # Modify the output file name as needed
    df.to_csv(output_file, index=False)
    print(f"Processed data saved to: {output_file}")

# Replace "/kaggle/input/bidi-reordered-dataset/bidi_reordered_attacked.csv" with your actual file path
dataset_path = "/kaggle/input/bidi-reordered-dataset/bidi_reordered_attacked.csv"
process_dataset(dataset_path)


100%|██████████| 5/5 [06:33<00:00, 78.72s/it]

Processed data saved to: processed_dataset.csv


In [21]:
data = pd.read_csv("/kaggle/working/processed_dataset.csv")
data.head(5)

,Reordered Text,OCR Result
0,Uga‭⁦‮⁦d⁩⁦n⁩‬⁩‬a‭⁦‮⁦s⁩⁦n⁩‬⁩‬ ‭⁦‮⁦‭⁦‮⁦ ⁩⁦d⁩‬⁩‬⁩...,LIOgQ f f€ fe FCF LF EL EEL EFL L LEP CFF FEF ...
1,‭⁦‮⁦‭⁦‮⁦I⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦t⁩⁦n⁩‬⁩‬⁩⁦E⁩‬⁩‬eb‭⁦‮...,NaN
2,‭⁦‮⁦‭⁦‮⁦e⁩⁦p⁩‬⁩‬⁩⁦‭⁦‮⁦i⁩⁦F⁩‬⁩‬‭⁦‮⁦t⁩⁦f⁩‬⁩‬y ⁩‬...,NaN
3,‭⁦‮⁦‭⁦‮⁦r⁩⁦b⁩‬⁩‬⁩⁦‭⁦‮⁦e⁩⁦C⁩‬⁩‬le⁩‬⁩‬a‭⁦‮⁦i⁩⁦t⁩...,\n\nTne fF CCFL KEL FFE FEL ELE LEE. FFE FE FE...
4,‭⁦‮⁦e⁩⁦N⁩‬⁩‬ver tru‭⁦‮⁦‭⁦‮⁦y⁩⁦h⁩‬⁩‬⁩⁦st ⁩‬⁩‬‭⁦...,\n\n\n\n
